# Tác vụ 5: Gọi mô hình Bedrock để tạo mã

Trong sổ tay này, bạn sẽ học cách sử dụng một mô hình ngôn ngữ lớn (LLM) để tạo mã dựa trên câu lệnh văn bản.

Câu lệnh dùng trong sổ tay này là câu lệnh không cần mẫu (zero-shot) vì chúng tôi không cung cấp bất kỳ văn bản ví dụ nào ngoài câu lệnh.

Để chứng minh khả năng tạo mã của các mô hình trong Amazon Bedrock, bạn sẽ dùng trường hợp sử dụng liên quan đến việc tạo mã. Bạn sẽ khám phá cách dùng ứng dụng Boto3 để giao tiếp với API Amazon Bedrock và cung cấp đầu vào cho API này, bao gồm tác vụ, hướng dẫn và dữ liệu đầu vào, để mô hình nền tạo ra đầu ra mà không cần cung cấp bất kỳ ví dụ bổ sung nào. Mục đích ở đây là để chứng minh cách các LLM mạnh mẽ có thể dễ dàng hiểu được tác vụ đang diễn ra và tạo đầu ra hấp dẫn.


## Kịch bản

Bạn đóng vai Moe, một Chuyên viên phân tích dữ liệu tại AnyCompany. Công ty muốn nắm được hiệu suất bán hàng của nhiều sản phẩm trong năm qua. Bạn đã được cung cấp một bộ dữ liệu có tên là sales.csv. Bộ dữ liệu chứa các cột sau:

- Định dạng ngày (YYYY-MM-DD)
- Product_ID (giá trị định danh duy nhất cho mỗi sản phẩm)
- Giá (giá bán mỗi sản phẩm)


Trong sổ tay này, bạn sẽ học cách tạo mã cho một câu lệnh nhất định. Bạn sử dụng Meta LLama 3 bằng API Amazon Bedrock với ứng dụng Boto3.

## Tác vụ 5.1: Thiết lập môi trường

Trong tác vụ này, bạn sẽ thiết lập môi trường của mình.


In [ ]:
#create a service client by name using the default session.
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))


## Tác vụ 5.2: Tạo mã

Trong tác vụ này, bạn sẽ chuẩn bị đầu vào cho dịch vụ Amazon Bedrock để tạo chương trình python cho trường hợp sử dụng của mình.

#### Thiết lập phòng thực hành – tạo dữ liệu sales.csv mẫu cho phòng thực hành này.



In [ ]:
# create sales.csv file
import csv

data = [
    ["date", "product_id", "price", "units_sold"],
    ["2023-01-01", "P001", 50, 20],
    ["2023-01-02", "P002", 60, 15],
    ["2023-01-03", "P001", 50, 18],
    ["2023-01-04", "P003", 70, 30],
    ["2023-01-05", "P001", 50, 25],
    ["2023-01-06", "P002", 60, 22],
    ["2023-01-07", "P003", 70, 24],
    ["2023-01-08", "P001", 50, 28],
    ["2023-01-09", "P002", 60, 17],
    ["2023-01-10", "P003", 70, 29],
    ["2023-02-11", "P001", 50, 23],
    ["2023-02-12", "P002", 60, 19],
    ["2023-02-13", "P001", 50, 21],
    ["2023-02-14", "P003", 70, 31],
    ["2023-03-15", "P001", 50, 26],
    ["2023-03-16", "P002", 60, 20],
    ["2023-03-17", "P003", 70, 33],
    ["2023-04-18", "P001", 50, 27],
    ["2023-04-19", "P002", 60, 18],
    ["2023-04-20", "P003", 70, 32],
    ["2023-04-21", "P001", 50, 22],
    ["2023-04-22", "P002", 60, 16],
    ["2023-04-23", "P003", 70, 34],
    ["2023-05-24", "P001", 50, 24],
    ["2023-05-25", "P002", 60, 21]
]

# Write data to sales.csv
with open('sales.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(data)

print("sales.csv has been created!")

## Tác vụ 5.3: Phân tích doanh số bán hàng với chương trình Python do Amazon Bedrock tạo ra

In [ ]:
# define prompt template
from langchain_core.prompts import PromptTemplate

def format_prompt(actor:str, input:str):
    match actor:
        case "user":
            prompt_template =  """<|begin_of_text|><|start_header_id|>{actor}<|end_header_id|>\n\n{input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n
"""
            prompt = PromptTemplate.from_template(prompt_template)
            return prompt.format(actor=actor,input=input)
        case _:
            print("requested actor >" + actor + "< is not supported")
            return ""    

In [ ]:
# Create the prompt
# Analyzing sales

prompt_data = """

You have a CSV, sales.csv, with columns:
- date (YYYY-MM-DD)
- product_id
- price
- units_sold

Create a python program to analyze the sales data from a CSV file. The program should be able to read the data, and determine below:

- Total revenue for the year
- Total revenue by product 
- The product with the highest revenue 
- The date with the highest revenue and the revenue achieved on that date
- Visualize monthly sales using a bar chart

Ensure the code is syntactically correct, bug-free, optimized, not span multiple lines unnessarily, and prefer to use standard libraries. Return only python code without any surrounding text, explanation or context.

"""
prompt=format_prompt("user",prompt_data)

In [ ]:
body = json.dumps({
    "prompt": prompt,
    "max_gen_len": 2048,
    "temperature": 0,
    "top_p": 1,
    }) 
    

## Tác vụ 5.4: Gọi mô hình

In [ ]:
modelId = "meta.llama3-8b-instruct-v1:0"
response = bedrock_client.invoke_model(body=body, modelId=modelId)
response_body = json.loads(response.get('body').read())
output_list = response_body.get("generation", [])
print(output_list)

##### <i aria-hidden="true" class="far fa-copy"></i> (Không bắt buộc) Sao chép mã đã tạo từ đầu ra đã in và chạy mã do Bedrock tạo trong ô dưới đây để xác thực.


Sau khi thử nghiệm việc sử dụng SDK `boto3`, bạn hiện đã nắm được những điểm cơ bản về API Amazon Bedrock. Nhờ sử dụng API này, bạn sẽ tạo được một chương trình python để phân tích và trực quan hóa dữ liệu bán hàng nhất định.



### Tự thực hiện

- Thay đổi câu lệnh theo trường hợp sử dụng cụ thể của bạn và đánh giá đầu ra của các mô hình khác nhau.
- Thử nghiệm với độ dài mã token để nắm được độ trễ và khả năng phản hồi của dịch vụ.
- Áp dụng các nguyên tắc tạo câu lệnh khác nhau để có được đầu ra tốt hơn.

### Dọn dẹp

Bạn đã hoàn thành sổ tay này. Để chuyển sang phần tiếp theo của phòng thực hành, hãy làm như sau:

- Đóng tệp sổ tay này và tiếp tục với **Tác vụ 6**.